Improve manTrack data handling.

The data handling of manTrack.

Essentially we need to maintain a table of x, y, r. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.patches as mpatch
import matplotlib.pyplot as plt

In [2]:
data = np.random.rand(5, 3)
df = pd.DataFrame(data=data, columns=["x","y","r"])
df

,x,y,r
0,0.538814,0.636296,0.465254
1,0.937663,0.632354,0.285311
2,0.236625,0.722360,0.185071
3,0.537219,0.590923,0.563008
4,0.071769,0.550058,0.612967


Two operations are necessary: add / delete a row. 

Adding a row can be achieved by the append method:

In [3]:
df2 = df._append({"x": 0,"y":1, "r": 2}, ignore_index=True)
df2

,x,y,r
0,0.538814,0.636296,0.465254
1,0.937663,0.632354,0.285311
2,0.236625,0.722360,0.185071
3,0.537219,0.590923,0.563008
4,0.071769,0.550058,0.612967
5,0.000000,1.000000,2.000000


To delete a row, we can use the drop method:

In [4]:
df3 = df2.drop(index=2)
df3

,x,y,r
0,0.538814,0.636296,0.465254
1,0.937663,0.632354,0.285311
3,0.537219,0.590923,0.563008
4,0.071769,0.550058,0.612967
5,0.000000,1.000000,2.000000


In [11]:
df4 = pd.concat([df3, pd.DataFrame(data={"x": 0,"y":1, "r": 2}, index=[df3.index.max()+1])]) 
df4

,x,y,r
0,0.538814,0.636296,0.465254
1,0.937663,0.632354,0.285311
3,0.537219,0.590923,0.563008
4,0.071769,0.550058,0.612967
5,0.000000,1.000000,2.000000
6,0.000000,1.000000,2.000000


When adding or deleting an item, we also need to modify the visibility and interactibility of the corresponding artist. 

When adding an element in the data, we also want to draw it on the canvas. This is done by add_patch. Note that we also need to make sure the new patch is interactive, by setting picker as True.

When removing an element, we only know which element to remove from the mouse click on artist. How do we know the correspondence between the artist we click on and the entry in the data table? The idea is to use the url property of artist. We set the index number as the url of an artist once we create it. Then when deleting it, we can trace back to the index number. 

In [28]:
circle1 = df.loc[1]
circ = mpatch.Circle((circle1.x, circle1.y), circle1.r)
circ.set_url(circle1.name)

We can get the index with get_url. 

In [32]:
circ.get_url()

1

Then we can delete the entry in the data table using the index.

In [33]:
ind = circ.get_url()
df4 = df3.drop(index=ind)
df4

,x,y,r
0,0.036202,0.319362,0.560386
3,0.511504,0.040327,0.850909
4,0.324577,0.226302,0.161794
5,0.000000,1.000000,2.000000


Circle 1 is deleted.

Lastly, we want to have the backward functionality, which is useful when we accidentally make undesired changes and want to revert it. 

To keep track of the action history, we can construct a history list of tuples:

```
history_list = [(artist1, "add"), (artist2, "delete"), ...]
```

Any adding or deleting actions will be appended to this list. When "backward" function is called, the last element will be popped.

In [35]:
history_list = [(circ, "add"), (circ, "delete")]

In [36]:
history_list.pop()

(<matplotlib.patches.Circle at 0x28a344f0af0>, 'delete')

In [37]:
history_list

[(<matplotlib.patches.Circle at 0x28a344f0af0>, 'add')]

With these ideas, I can rework the manTrack software to make it more robust in the data handling. 

Some user friendly features should also be considered:

1. capability of zooming in
2. visible circle when mouse pressed